In [1]:
from pymongo import MongoClient
from tqdm import tqdm
import pandas as pd
from pd_doc2vec import doc2vec



client = MongoClient()
client = MongoClient('da1.eecs.utk.edu', 27017) 
#Connects to the MongoDB, make sure youre SSH'ed into the docker

collection_list = client['twitter'].collection_names() 
#Scrapes all the collection names from the Docker
print(collection_list)

['LabeledStatuses_Sandy_K', 'Users_Maria', 'Statuses_Maria_A', 'Users_MiscNews', 'Users_Irma', 'LabeledStatuses_Irma_A_', 'Statuses_Irma_C', 'LabeledStatuses_MiscTechCompanies_A_', 'Statuses_Florence_A', 'LabeledStatuses_Irma_C', 'Users_Florence', 'Misc_FilmReviews', 'Statuses_Florence_C', 'Statuses_Irma_A', 'Statuses_MiscPower_A', 'Geolocations_Irma', 'Statuses_Maria_C', 'LabeledStatuses_MiscTechCompanies_C', 'Statuses_MiscClimateChange_A', 'Users_MiscByCategory', 'LabeledStatuses_MiscRelevant_C', 'Statuses_Irma_K']


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  if sys.path[0] == '':


In [2]:
collection_list = [x for x in collection_list if "Maria" not in x and "User" not in x and  "Geolocations" not in x and "Sandy" not in x and "Statuses" in x and "Tech" not in x and "Film" not in x and "Misc" not in x  and "User" not in x and "_K" not in x   ] 
#Scrapes all the collection names from the Docker
print(collection_list)

['LabeledStatuses_Irma_A_', 'Statuses_Irma_C', 'Statuses_Florence_A', 'LabeledStatuses_Irma_C', 'Statuses_Florence_C', 'Statuses_Irma_A']


In [3]:
i = 0
sentimentdata = []
for posts in (collection_list):
    i = i+1
    print("Processing", posts, i, "/", str(len(collection_list)))
    for post in tqdm(client['twitter'][posts].find({})):  #{'sentiment': {"$exists": True}}
        #Extracts all the entries with a sentiment field
        post.update({"Source": str(posts)}) 
        #Updates each data with the name of the collection it comes from
        sentimentdata.append(post) 

0it [00:00, ?it/s]

Processing LabeledStatuses_Irma_A_ 1 / 6


848it [00:00, 3565.68it/s]
0it [00:00, ?it/s]

Processing Statuses_Irma_C 2 / 6


763964it [00:50, 15127.52it/s]
0it [00:00, ?it/s]

...

24823it [00:08, 2968.74it/s]
1000it [00:00, 22448.64it/s]
0it [00:00, ?it/s]

Processing LabeledStatuses_Irma_C 4 / 6
Processing Statuses_Florence_C 5 / 6


24286it [00:04, 5844.84it/s]
0it [00:00, ?it/s]

Processing Statuses_Irma_A 6 / 6


556482it [02:51, 3243.12it/s]


In [4]:
        
df = pd.DataFrame(sentimentdata)
# pulls data into the Pandas DataFrame

print(df.head(3))

                    Source                       _id categories code  \
0  LabeledStatuses_Irma_A_  5bed635596c339521e3cf088        NaN    2   
1  LabeledStatuses_Irma_A_  5bed635596c339521e3cf089        NaN    2   
2  LabeledStatuses_Irma_A_  5bed635596c339521e3cf08a        NaN    4   

   contributors coordinates          created_at  \
0           NaN        None 2017-09-11 11:20:21   
1           NaN        None 2017-09-11 06:10:46   
2           NaN        None 2017-09-11 00:25:45   

                                            entities  \
0  {'symbols': [], 'urls': [], 'user_mentions': [...   
1  {'symbols': [], 'urls': [], 'user_mentions': [...   
2  {'symbols': [], 'urls': [], 'user_mentions': [...   

                                      extended_tweet  favorite_count  \
0  {'display_text_range': [11, 81], 'full_text': ...             0.0   
1  {'display_text_range': [0, 119], 'full_text': ...             0.0   
2  {'display_text_range': [11, 58], 'full_text': ...             

In [5]:
from tqdm import tqdm
import pandas as pd
from pd_doc2vec import doc2vec

emoset = pd.read_csv("987.csv")
emoset.fillna(0, inplace=True)
emoset = emoset.applymap(str)
emoset = emoset[emoset.Opinion != "\n"]


x = doc2vec(emoset, "Tweet", ["Opinion",  "Emotion", 'Sarcasm', 'Sentiment', 'Emotion class'])


x.score(verbose=True)



x2 = doc2vec(emoset, "Tweet", ["Emotion",  "Opinion", 'Sarcasm', 'Sentiment', 'Emotion class'])


x2.score(verbose=True)


  0%|          | 0/50 [00:00<?, ?it/s]/usr/local/lib/python3.5/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
100%|██████████| 50/50 [00:00<00:00, 68.57it/s]
/home/nwest13/Curent/Manny/pd_doc2vec.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  test['results'] = self.predict(test[X])


Accuracy Score:  0.74
Label Score:  [0.8395061728395061, 0.3157894736842105]


100%|██████████| 50/50 [00:00<00:00, 65.67it/s]


Accuracy Score:  0.7
Label Score:  [0.761904761904762, 0.5945945945945946]


[[0.761904761904762, 0.5945945945945946], 0.7]

In [6]:
df.to_csv("corpus_rand.csv", index=False)

In [7]:
df = df.sample(5000)

In [8]:
df["emotion_ml"] = x.predict(df["text"])
df["opinion_ml"] = x2.predict(df["text"])


  0%|          | 0/5000 [00:00<?, ?it/s]/usr/local/lib/python3.5/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
100%|██████████| 5000/5000 [00:31<00:00, 160.51it/s]


In [9]:
df.to_csv("corpus_sample.csv", index=False)